## Game of life represenation using dask

This assignment used the initial, synchronous implementation of the game of life from one of the earlier tutorials as a base. The code was adapted for dask, and the result was analysed between various parameters and computation methods. Furthermore, an alternative implementation of the game of life is presented. The alternative version uses convolve from Scipy to get the number of the cell's neighbours. The data used for the analysis comes from the files 1000x1000.0.1 and 10000x10000.0.2. 

**Basic implemenation**  
The basic implementation can be found in code/game.py. The implementation of the game logic is in the block below, and the helper functions for reading from the file, writing from file and getting the arguments are in the appendix.

In [ ]:
def neighbors_number(board, row, col):
    # add plus two to create a proper iteration
    neighbors = board[max(0, row-1):min(board.shape[0], row+2), max(0, col-1):min(board.shape[1], col+2)]
    return np.sum(neighbors) - board[row, col]

def play_game(board, iterations):
    for i in range(iterations):
        new_board = board.copy()
        for r in range(board.shape[0]):
            for c in range(board.shape[1]):
                #for dead cell
                if board[r][c] == 0 and neighbors_number(board, r, c) == 3:
                    new_board[r][c] = 1
                #for alive cell
                if board[r][c] == 1 and (neighbors_number(board, r, c) < 2 or neighbors_number(board, r, c) > 3):
                    new_board[r][c] = 0
        board = new_board
        print(board)
    return board

For the dask implementation, the board that is being passed to play_game is already a dask array, and the rest of the implementation is as follows:

In [ ]:
def neighbors_number(board, row, col):
    # add plus two to create a proper iteration
    neighbors = board[max(0, row-1):min(board.shape[0], row+2), max(0, col-1):min(board.shape[1], col+2)]
    return np.sum(neighbors) - board[row, col]

def tick(board):
    # the .copy from the original code takes a lot of space change to empty array
    w, h = board.shape
    new_board = np.zeros((w, h),dtype=np.uint8)
    for r in range(w):
        for c in range(h):
            #for dead cell
            if board[r][c] == 0 and neighbors_number(board, r, c) == 3:
                new_board[r][c] = 1
            #for alive cell
            if board[r][c] == 1 and (neighbors_number(board, r, c) < 2 or neighbors_number(board, r, c) > 3):
                new_board[r][c] = 0
    return new_board

def play_game(dask_board, iterations):
    for i in range(iterations):
        dask_board = dask_board.map_overlap(tick, depth=1, boundary='none')
    final_board = dask_board.compute()
    return final_board

**Paramters analysis**  

_note: the programs are only time for how long it takes to do one iteration; the read and write functions are excluded from the total time_  

The default dask client was first run with  'processes' and then 'threads' argument for the scheduler to determine which is better. 
  
The result below show that for this task the 'processes' option run faster in both wall and CPU time. 

|           | Processes | Threads |
|-----------|-----------|---------|
| Wall time | 4.66      | 19.7    |
| CPU time  | 1.12      | 18.4    | 

<p align="center">
  <img src="images/CPUvsWorkers.png" alt="Time vs number of workers">
</p>  

There is a trend visible that with an increasing number of workers, the CPU time increases while the wall time decreases. A decrease in the wall time is typical for parallel computing as the task can be done simultaneously and not one after another. The increase in the CPU time can be explained by the fact that with the increasing number of workers, the total time they spent on the task increased. Might be bigger than in the case of one worker.  

Similar operation for the best case of 6 workers was done for number of threads to see how the time changes:  

<p align="center">
  <img src="images/CPUvsThreads.png" alt="Time vs number of threads">
</p> 


This shows that the best possible cobniation can wall time below 5 second and the CPU time around 1 second. The standard run of the game of life had both wall and CPU time of 14.5 seconds which shows that using dask is beneficial in this case.

## Apendix

Reading from the file

In [ ]:
#argument chunk_size only for dask
def read_input_file(input_file, chunk_size):
    # initialize board
    with open(input_file) as f:
        w, h = [int(x) for x in next(f).split()]
        # use the smallest data type
        board = np.zeros((w, h),dtype=np.uint8)
        for line_count, line in enumerate(f, start=0):
            single_numbers = line.split()
            x, y = map(int, single_numbers[:2])
            board[x][y] = 1
        #this line only for dask and return dask_board in that case
        dask_board = da.from_array(board, chunks=(chunk_size, chunk_size))
    return dask_board

Writting to file

In [ ]:
def write_output_file(output, board):
    w = board.shape[0]
    h = board.shape[1]
    live_cells = []
    for r in range(w):
        for c in range(h):
            if board[r][c] == 1:
                live_cells.append([r, c])
    f = open(output, "w")
    f.write(str(w)+ " "+ str(h)+"\n")
    for cell in live_cells:
        f.write(str(cell[0])+ " "+ str(cell[1])+ "\n")
    f.close()

Reading the input

In [ ]:
def main():
    try:
        input_name = sys.argv[1]
    except IndexError:
        sys.exit("No output filename")
    try:
        output_name= sys.argv[2]
    except IndexError:
        sys.exit("No output filename")
    try:
        n = int (sys.argv[3])
    except:
        sys.exit(f"Entered value is not a nuber {sys.argv[3]}")
    try:
        chunk_size = int(sys.argv[4])
    except:
        sys.exit(f"Entered value is not a nuber {sys.argv[4]}") 
    
    dask.config.set(scheduler='processes')
    client = Client()   
    board = read_input_file(input_name, chunk_size)
    final_board = play_game(board, n)
    write_output_file(output_name, final_board)
    client.shutdown()